In [1]:
!pip install transformers datasets sentence-transformers faiss-cpu scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 49.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset
dataset = load_dataset("tweet_eval", "sentiment")
print(dataset['train'].features['label'])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sentiment/train-00000-of-00001.parquet:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

sentiment/test-00000-of-00001.parquet:   0%|          | 0.00/901k [00:00<?, ?B/s]

sentiment/validation-00000-of-00001.parq(…):   0%|          | 0.00/167k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

ClassLabel(names=['negative', 'neutral', 'positive'])


In [4]:
from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128)

train_ds = dataset['train'].map(preprocess, batched=True)
val_ds = dataset['validation'].map(preprocess, batched=True)
test_ds = dataset['test'].map(preprocess, batched=True)
train_ds = train_ds.rename_column("label", "labels")
val_ds = val_ds.rename_column("label", "labels")
test_ds = test_ds.rename_column("label", "labels")
train_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
val_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/45615 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall":recall, "f1": f1}

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/sentiment_model", eval_strategy="epoch", save_strategy="epoch", learning_rate=2e-5,
                                  per_device_train_batch_size=16, per_device_eval_batch_size=64, num_train_epochs=3, weight_decay=0.01, load_best_model_at_end=True,
                                  logging_dir="/content/drive/MyDrive/logs")
trainer = Trainer(model=model,args=training_args, train_dataset=train_ds, eval_dataset=val_ds, tokenizer=tokenizer, compute_metrics=compute_metrics)
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-586121043.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model,args=training_args, train_dataset=train_ds, eval_dataset=val_ds, tokenizer=tokenizer, compute_metrics=compute_metrics)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mariamyasir2002 (mariamyasir2002-misr-higher-institute-of-engineering-and) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.622400,0.623524,0.727000,0.741022,0.727000,0.726855
2,0.496500,0.651838,0.735500,0.735154,0.735500,0.731813
3,0.371300,0.701548,0.733000,0.732830,0.733000,0.732841


TrainOutput(global_step=8553, training_loss=0.5156158927615088, metrics={'train_runtime': 1799.7285, 'train_samples_per_second': 76.036, 'train_steps_per_second': 4.752, 'total_flos': 4531956111832320.0, 'train_loss': 0.5156158927615088, 'epoch': 3.0})

In [7]:
trainer.evaluate(test_ds)

{'eval_loss': 0.6521884799003601,
 'eval_accuracy': 0.7050634972321719,
 'eval_precision': 0.7089767375750919,
 'eval_recall': 0.7050634972321719,
 'eval_f1': 0.7036175143751151,
 'eval_runtime': 37.5479,
 'eval_samples_per_second': 327.156,
 'eval_steps_per_second': 5.113,
 'epoch': 3.0}

In [23]:
trainer.save_model("/content/drive/MyDrive/sentiment_model")
tokenizer.save_pretrained("/content/drive/MyDrive/sentiment_model")
model = model.to("cpu")
model.save_pretrained("/content/drive/MyDrive/sentiment_model_cpu")
tokenizer.save_pretrained("/content/drive/MyDrive/sentiment_model_cpu")

('/content/drive/MyDrive/sentiment_model_cpu/tokenizer_config.json',
 '/content/drive/MyDrive/sentiment_model_cpu/special_tokens_map.json',
 '/content/drive/MyDrive/sentiment_model_cpu/vocab.txt',
 '/content/drive/MyDrive/sentiment_model_cpu/added_tokens.json',
 '/content/drive/MyDrive/sentiment_model_cpu/tokenizer.json')

In [9]:
from transformers import pipeline

clf = pipeline("text-classification", model="/content/drive/MyDrive/sentiment_model", tokenizer="/content/drive/MyDrive/sentiment_model")
print(clf("I love this airline, great service!"))

Device set to use cuda:0


[{'label': 'LABEL_2', 'score': 0.9877969622612}]


In [20]:
import faiss, pickle
from sentence_transformers import SentenceTransformer

train_texts = dataset['train']['text']
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(train_texts, show_progress_bar=True, convert_to_numpy=True)
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)
faiss.write_index(index, "faiss_index.index")
train_texts = list(dataset['train']['text'])
with open("passages.pkl", "wb") as f:
    pickle.dump(train_texts, f)

Batches:   0%|          | 0/1426 [00:00<?, ?it/s]

In [21]:
def retrieve_passages(query, k=5):
    q_emb = embedder.encode([query])
    D, I = index.search(q_emb, k)
    return [train_texts[int(i)] for i in I[0]]

In [22]:
print(retrieve_passages("The flight was terrible", k=3))

["So sad to hear about Pavol Demitra & the other KHL players' tragic passing in the plane crash today. Absolutely terrible news.", '@user get to Manchester at 7 tonight just sat in Milan airport now ur flight go okay ?', 'Flight starring Denzel Washington was a good ass movie go see it tomorrow when it comes out']


In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

GEN_MODEL = "google/flan-t5-small"
gen_tok = AutoTokenizer.from_pretrained(GEN_MODEL)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [18]:
def explain_sentiment(text, predicted_label, k=3):
    retrieved = retrieve_passages(text, k=k)
    prompt = f"""
    Input: "{text}"
    Predicted sentiment: {predicted_label}
    Retrieved examples:
    - {"\n- ".join(retrieved)}
    Explain in 1-2 sentences why the prediction makes sense based on the retrieved evidence.
    """
    inputs = gen_tok(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = gen_model.generate(**inputs, max_length=80)
    explanation = gen_tok.decode(outputs[0], skip_special_tokens=True)
    return {"retrieved": retrieved, "explanation": explanation}

In [19]:
sample = "The seats were uncomfortable and the service was rude."
pred = clf(sample)[0]['label']
rag_output = explain_sentiment(sample, pred)
print("Prediction:", pred)
print("Retrieved passages:", rag_output['retrieved'])
print("Explanation:", rag_output['explanation'])

Prediction: LABEL_0
Retrieved passages: ["Walked backstage and behind the bandstand I sat on Paul McCartney's amp cases where security told me to sit while the Wings began the show.", 'it was my first day on the job and I may or may not have served David Beckham coffee.', '@user Norma they are cat 1 bought on 1st day they released. Cant tell exact seats till 2 morrow in work computer from Ticketek.']
Explanation: negative
